- Reference : https://cpm0722.github.io/pytorch-implementation/transformer

# 1. Transformer

- Transformer의 개괄적인 구조

In [ ]:
# encoder 구현
# decoder 구현

class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x, z): # (?) x, z 는 각각 뭘까?
        c = self.encoder(x)
        y = self.decoder(z, c)
        return y

# 2. Encoder

- Encoder

In [ ]:
# Encoder layer에서 input과 output의 shape는 동일하다 (그렇게 설계해야만 함)
# input sentence 문장의 embedding (= 전체 Encoder의 input) -> Layer 1을 통과한 output -> Layer 2를 통과 
# -> Layer 3을 통과 -> ... -> Layer N을 통과 -> context (= 전체 Encoder의 output)
# 논문에서는 N = 6 사용
# Encoder를 여러 Layer 겹쳐 쌓는 이유?
# 각 Encoder Layer의 역할은, input으로 들어오는 vector에 대해 더 높은 차원(더 넓은 관점, 더 추상적인 정보로)에서의 context를 담는 것
# 겹겹이 쌓이면서 점저 더 높은 차원의 context가 저장되게 됨


class Encoder(nn.Module):
    def __init__(self, encoder_layer, n_layer): # n_layer: Encoder Layer의 개수
        super(Encoder, self).__init__()
        # (?) 이 부분 nn.ModuleList(?) 써서 개선할 수 있지 않을까
        self.layers = []
        for i in range(n_layer):
            self.layers.append(copy.deepcopy(encoder_layer))

          
    def forward(self, x):
        out = x # 초기화 후 반복 (= Encoder 전체의 input)
        for layer in self.layers:
            out = layer(out)
        return out # (= Encoder 전체의 output (: context) )

- Encoder Layer

In [ ]:
# Encoder Layer 는 크게 Multi-Head Attention Layer 과 Position-wise-Feed-Forward Layer 로 구성됨

class EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
        super(EncoderLayer, self).__init__()
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer

    def forward(self, x):
      out = self.multi_head_attention_layer(x)
      out = self.position_wise_feed_forward_layer(out)
      return out

- Self-Attention

  (= 논문에서는 'Scaled-Dot-Attention'이라고 표현함)

In [ ]:
# 실제 model에 들어오는 input은 한 개의 문장이 아니라 mini-batch이기 때문에 Q, K, V의 shape가 n_batch×seq_len×dk 이다 !!

def calculate_attention(self, query, key, value, mask):
    # query, key, value's shape: (n_batch, seq_len, "d_k")
    # (query, key, value는 서로 다른 FC Layer를 거쳐 n_batch×max_seq_len×dk로 변형되었다.)

    # mask : pad mask matrix (값이 0: pad token, 값이 1: 문장 내 기존 토큰)
    # (pad mask matrix는 Transformer 외부 (대개 Batch class)에서 생성되어 Transformer에 인자로 들어오게 된다.)

    # 준비 단계에서 FC_Layer를 통해 구한 Key -> 의 (행, 열) 크기 중 '열'이 dimension_k
    d_k = key.size(-1) # (?) key.shape[-1] 로 써도 같은 값이 나올까??

    # Q x K^T, attention_score's shape: (n_batch, seq_len, seq_len)
    attention_score = torch.matmul(query, key.transpose(-2, -1))

    # scaling
    attention_score = attention_score / math.sqrt(d_k)

    # (Optional) Pad Masking
    if mask is not None:
        attention_score = score.masked_fill(mask == 0, -1e9) # (?)score가 아니라 attention_score 아닌가?

    # softmax, attention_prob's shape: (n_batch, seq_len, seq_len)
    attention_prob = F.softmax(score, dim=-1)

    # Attention_Prob x V, out's shape: (n_batch, seq_len, d_k)
    out = torch.matmul(attention_prob, value)

    return out # (n_batch, seq_len, d_k)


- Multi-Head Attention Layer

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, d_model, h, qkv_fc_layer, fc_layer):
        # qkv_fc_layer's shape : (d_embed, "d_model")
        #                                   => (중요) 단순히 d_k 가 아닌, d_model 로 Q,K,V 를 생성해서
        #                                            -> 한 번의 self-attention 연산으로 output 계산이 가능하도록 만든다!
        # fc_layer's shape : (d_model, d_embed)

        super(MultiHeadAttentionLayer, self).__init__()

        self.d_model = d_model
        self.h = h

        # [1] Q, K, V 생성을 위한 FC Layer 선언 (d_embed, d_model)
        # (deepcopy 를 사용해 -> 3가지가 각 서로 다른 weight를 갖고 별개로 연산될 수 있도록 복사하여 선언함)
        self.query_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.key_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.value_fc_layer = copy.deepcopy(qkv_fc_layer)

        # [2] 멀티헤드 attention 계산 이후, 거쳐가는 FC Layer (d_model, d_embed)
        self.fc_layer = fc_layer


    # (+) Decoder Layer에서는 query, key, value 가 서로 다른 embedding에서 넘어온다 !
    #     따라서, 인자로 query, key, value를 모두 별개로 인풋해 주는 것
    def forward(self, query, key, value, mask=None):
        # query, key, value's shape : (n_batch, seq_len, d_embed)
        # (<- Input Sentence의 Q, k, V 차원)
        # mask's shape : (n_batch, seq_len, seq_len)

        n_batch = query.shape[0] # get n_batch

        
        # FC Layer 통과하면서 + 차원 변형해줌
        def transform(x_input, fc_layer_to_transform): # reshape (n_batch, seq_len, d_embed) to (n_batch, seq_len, d_k)
            out = fc_layer_to_transform(x_input)
            # out's shape : (n_batch, seq_len, d_model)

            out = out.view(n_batch, -1, self.h, self.d_model//self.h)
            # out's shape : (n_batch, seq_len, "h", d_k)
            # ("view" 함수의 사용용도 : 붙어있는 차원을 떼어낼 때 쓴다)
            # ("-1" 값의 의미 : 자동으로 변환이 가능한 차원이 지정되어 차원 배정이 이루어짐 -> 2번째 차원 "seq_len"은 자동 지정된 것임)

            out = out.transpose(1, 2).contiguos()
            # out's shape : (n_batch, "h", seq_len, d_k)
            # (?) "view" 함수는 contiguous한 tensor를 반환하고, "transpose" 함수는 non-contiguous한 tensor를 반환하는데,
            #     여기서 코드를 "transpose(1, 2).contiguous()"로 안 써주어도 되나??
            #     (다음에 이어서 view 등 함수를 쓸 일이 있으면 이렇게 해주는게 안전한데, 앞으로 또 연산할 일이 잘 없어서 그냥 패스한 것 ..?)

            return out
            # 이러한 차원 transform 작업을 수행하는 이유는 ?
            # : 위에서 작성했던 "calculate_attention()"이 input으로 받고자 하는 shape가
            #   "(n_batch×...×seq_len×dk)"이기 때문이다.



      query = transform(query, self.query_fc_layer) # [1] Q, K, V 생성을 위한 FC Layer
      key = transform(key, self.key_fc_layer) # [1] Q, K, V 생성을 위한 FC Layer
      value = transform(value, self.key_fc_layer) # [1] Q, K, V 생성을 위한 FC Layer


      if mask not None:
          # 이전 mask's shape : (n_batch, seq_len, seq_len)
          mask = mask.unsqueeze(1)
          # 결과 mask's shape : (n_batch, "1", seq_len, seq_len)
      
      # (?) 아래에서 시행하는 "calculate_attention" 함수 내에서 "어텐션 스코어"를 구할때 mask_fill 함수를 수행하는 부분이 있음
      #      그런데, 왜 mask 인자를 인풋하기 이전에 차원을 늘려서 바꾸어주어야만 할까 ?
      #         : (n_batch×seq_len×seq_len) 형태를 (n_batch×1×seq_len×seq_len)로 변경하게 된다.
      #           이는 calculate_attention() 내에서 masking을 수행할 때 broadcasting이 제대로 수행되게 하기 위함이다.
      #           -> transform 함수와 마찬가지로 위에서 작성했던 "calculate_attention()"이 input으로 받고자 하는 shape가
      #               "(n_batch×...×seq_len×dk)"이기 때문이다. (?)


      out = self.calculate_attention(query, key, value, mask)
      # out's shape: (n_batch, h, seq_len, d_k)

      out = out.transpose(1, 2)
      # out's shape: (n_batch, seq_len, h, d_k)

      out = contiguous().view(n_batch, -1, self.d_model)
      # out's shape: (n_batch, seq_len, "d_model")
      #                                 * d_model 값 = (h * d_k)
      # 차원 합쳐주는 이유?
      #     => "멀티-헤드" 어텐션이기 때문에 ?
      #       h(=8) 만큼 dimension을 합쳐서 Q,K,V vector를 생성해주어야 -> 한 번의 self-attention으로 연산이 수행 가능하다 !
      #       (h(=8) 만큼의 연산 효과를 수행하는 이유는, 여러 "attention" 정보를 반영함으로써, 더 많은 "정보"를 담아내기 위함이다 !)

      out = self.fc_layer(out) # [2] 멀티헤드 attention 계산 이후, 거쳐가는 FC Layer
                                # (d_model -> d_embed 로 차원 변경의 효과 있음)
      # out's shape: (n_batch, seq_len, "d_embed")






# Input Sentence의 Q, k, V 차원 : ( n_batch, seq_len, d_embed )
# -> [1] Q, K, V 생성을 위한 FC Layer 통과 : (d_embed -> d_model 변환시키는 weight matrix임)
# 그 결과, 생성된 Q, K, V 의 크기는 : ( n_batch, seq_len, d_k )

# <멀티헤드 어텐션> 수행 ...

# -> [2] 멀티헤드 attention 계산 이후, 거쳐가는 FC Layer 통과 (d_model -> d_embed 로 다시 변환시키는 weight matrix임)
# ("멀티 헤드 어텐션" 전체 연산(즉, 하나의 큰 함수)에서의 input과 output 크기를 맞춰주기 위함)

- Encoder Layer (수정)

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
      super(EncoderLayer, self).__init__()
      self.multi_head_attention_layer = multi_head_attention_layer
      self.position_wise_feed_forward_layer = position_wise_feed_forward_layer


  def forward(self, x, mask): # mask 인자 추가
      out = self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask) # 입력 인자 : (x, x, x, mask)
      # 앞의 multi_head_attention_layer 에서는 인자가 1개(x)일 것으로 가정하고 코드를 구현하였음
      # but, 실제로는 query, key, value를 받아야하므로 이를 수정해준다 ! + mask 역시 인자로 받는다 !
      out = self.position_wise_feed_forward_layer(out)
      return out

- Encoder (수정)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, encoder_layer, n_layer): # n_layer: Encoder Layer의 개수
        super(Encoder, self).__init__()
        # (?) 이 부분 nn.ModuleList(?) 써서 개선할 수 있지 않을까
        self.layers = []
        for i in range(n_layer):
            self.layers.append(copy.deepcopy(encoder_layer))

          
    def forward(self, x, mask): # mask 인자 추가
        out = x # 초기화 후 반복 (= Encoder 전체의 input)
        for layer in self.layers:
            out = layer(out, mask)
        return out # (= Encoder 전체의 output, context)

- Transformer (수정)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x, z): # (?) x, z 는 각각 뭘까?
                                  # x : src
                                  # z : trg
        encoder_output = self.encoder(x, mask)
        out = self.decoder(z, encoder_output)
        return out

- Position-wise Feed Forward Layer

In [ ]:
# Feed Forward Layer는 단순하게 2개의 FC Layer를 갖는 레이어

class PositionWiseFeedForwardLayer(nn.Module):
    def __init__(self, first_fc_layer, second_fc_layer):
        self.first_fc_layer = first_fc_layer
        self.second_fc_layer = second_fc_layer


    def forward(self, x):
      out = self.first_fc_layer(x)
      # 첫 번째 FC Layer의 weight matrix 크기 : (d_embed x d_ff)
      out = F.relu(out)
      out = self.dropout(out) # (이 과정에서 dropout 도 추가)
      out = self.second_fc_layer(out)
      # 두 번째 FC Layer의 weight matrix 크기 : (d_ff x d_embed)
      return out # 이를 통해 input과 같은 shape으로 output의 shape 유지가 가능

- Norm Layer (Residual Connection)

In [ ]:
class ResidualConnectionLayer(nn.Module):
    def __init__(self, norm_layer):
        super(ResidualConnectionLayer, self).__init__()
        self.norm_layer = norm_layer


    def forward(self, x, sub_layer):
        out = sub_layer(x) + x # Residual Connection 적용
        out = self.norm_layer(out) # Layer Normalization 추가 적용
        return out


- Encoder Layer (수정 2)

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
      super(EncoderLayer, self).__init__()
      self.multi_head_attention_layer = multi_head_attention_layer
      self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
      self.residual_connection_layers = [ResidualConnectionLayer(copy.deepcopy(norm_layer)) for i in range(2)]
      # residual_connection_layers에 ResidualConnectionLayer를 2개 생성해 저장하고,


  def forward(self, x, mask): # mask 인자 추가
      '''
      out = self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask) # 입력 인자 : (x, x, x, mask)
      # 앞의 multi_head_attention_layer 에서는 인자가 1개(x)일 것으로 가정하고 코드를 구현하였음
      # but, 실제로는 query, key, value를 받아야하므로 이를 수정해준다 ! + mask 역시 인자로 받는다 !
      out = self.position_wise_feed_forward_layer(out)
      '''

      # 0번째는 multi_head_attention_layer를 감싸고, 1번째는 position_wise_feed_forward_layer를 감싸게 된다.
      # <ResidualConnectionLayer 클래스의 forward 함수의 인자> 는
      # forward(self, x, sub_layer)
      out = self.residual_connection_layers[0](x, lambda x: self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask))
      out = self.residual_connection_layers[1](x, lambda x: self.position_wise_feed_forward_layer(x))
      # (?) 헷갈림
      return out

# 3. Decoder

- Teacher Forcing in Transformer (Subsequent Masking)

`make_std_mask()는 subsequent_mask()를 호출해 subsequent mask을 생성하고, 이를 pad mask와 결합한다. 위의 code는 Transformer 내부가 아닌 Batch class 내에서 실행되는 것이 바람직할 것이다. mask 생성은 Transformer 내부 작업이 아닌 전처리 과정에 포함되기 때문이다. 따라서 Encoder에 적용되는 pad mask와 동일하게 Batch class 내에서 생성될 것이다.`

In [ ]:
# Transformer가 RNN에 비해 갖는 가장 큰 장점은 병렬 연산이 가능하다는 것이었다.
# 병렬 연산을 위해 ground truth의 embedding을 matrix로 만들어 input으로 그대로 사용하게 되면,
# Decoder에서 Self-Attention 연산을 수행하게 될 때 현재 출력해내야 하는 token의 정답까지 알고 있는 상황이 발생한다.
# 따라서 masking을 적용해야 한다. i번째 token을 생성해낼 때, 1∼i−1의 token은 보이지 않도록 처리를 해야 하는 것이다.


def subsequent_mask(size):
    atten_shape = (1, size, size)
    mask = np.triu(np.ones(atten_shape), k=1).astype('uint8') # masking with upper triangle matrix
    return torch.from_numpy(mask)==0 # reverse (masking=False, non-masking=True)


def make_std_mask(tgt, pad): # subsequent_mask()를 호출해 subsequent mask을 생성하고, 이를 pad mask와 결합한다.
    tgt_mask = (tgt != pad) # pad masking
    tgt_mask = tgt_mask.unsqueeze(-2) # reshape (n_batch, seq_len) -> (n_batch, 1, seq_len)
    tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1))).type_as(tgt_mask.data) # pad_masking & subsequent_masking
    return tgt_mask
    # (??)



- Transformer (수정 2)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    # 기존에는 Encoder에서 사용하는 pad mask(src_mask)만이 forward()의 인자로 들어왔다면,
    # 이제는 Decoder에서 사용할 subsequent mask (trg_mask)도 함께 주어진다.

    # forward(인코더의 input, 디코더의 input, 인코더의 mask, 디코더의 mask)
    # 인코더의 mask : 'src_mask' 인자 (= 일반적인 Encoder에서 사용하는 pad mask)
    # 디코더의 mask : 'trg_mask' 인자 (= Decoder에서 사용할 subsequent mask)
    def forward(self, src, trg, src_mask, trg_mask): # (?) x, z 는 각각 뭘까?
                                  # x : src
                                  # z : trg
        encoder_output = self.encoder(src, src_mask) # src_mask 가 함께 인풋됨
        out = self.decoder(trg, trg_mask, encoder_output) # trg_mask 가 함께 인풋됨
        return out

- Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, sub_layer, n_layer): # n_layer: Decoder Layer의 개수
        super(Decoder, self).__init__()
        # (?) 이 부분 nn.ModuleList(?) 써서 개선할 수 있지 않을까
        self.layers = []
        for i in range(n_layer):
            self.layers.append(copy.deepcopy(sub_layer))

          
    def forward(self, x, mask, encoder_output, encoder_mask): # encoder_mask 와 encoder_mask 추가
        # x : Decoder의 input sentence
        # mask : Decoder의 subsequent mask
        # encoder_output : Encoder의 Context
        # encoder_mask : Encoder의 mask(: 일반적인 encoder에서 사용한 pad masking)

        out = x # 초기화 후 반복 (= Encoder 전체의 input)
        for layer in self.layers:
            out = layer(out, mask, encoder_output, encoder_mask) # encoder_mask 와 encoder_mask 추가
        return out # (= Decoder 전체의 output -> 이것은 무엇인가??)

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, masked_multi_head_attention_layer, multi_head_attention_layer, position_wise_feed_forward_layer, norm_layer):
      super(DecoderLayer, self).__init__()
      self.masked_multi_head_attention_layer = ResidualConnectionLayer(masked_multi_head_attention_layer, copy.deepcopy(norm_layer))
      self.multi_head_attention_layer = ResidualConnectionLayer(multi_head_attention_layer, copy.deepcopy(norm_layer))
      self.position_wise_feed_forward_layer = ResidualConnectionLayer(position_wise_feed_forward_layer, copy.deepcopy(norm_layer))

      # (?) <ResidualConnectionLayer 클래스의 forward 함수의 인자> 는
      # forward(self, x, sub_layer)
      # 어떻게 x 값에 layer를 인풋해도 되는 것 ?????



  def forward(self, x, mask, encoder_output, encoder_mask): # mask 인자 추가
      # 디코더는 '멀티-헤드 어텐션 레이어' 2개 + 'position-wise-feed-forward layer' 1개로 이루어져 있음

      # (1) 1번 째 멀티-헤드 어텐션 레이어
      
      # <특징>
      # "subsequent mask"가 적용된다.
      # self-attention 을 수행하는 원리이다.
      out = self.masked_multi_head_attention_layer(query=x, key=x, value=x, mask=mask)

      # (2) 2 번 째 멀티-헤드 어텐션 레이어
      
      # <특징>
      # 이전 레이어인 'masked-multi-head-attention layer'로부터 넘겨 받은 ouput => query로 쓰임 (teacher forcing 원리가 들어감)
      # + Encoder에서 도출된 context (=encoder_output) => key, value로 쓰임
      # self-attention이 아닌, 이 두 가지 input 사이의 attention을 계산하는 단계이다.

      # (?) "일반적인 pad masking (= 'encoder_output' 인자)" 을 쓰는 이유 ?
      #     : teacher forcing으로 넘어온 input(= 원하는 output과 유사함)을 query로 날려서
      #       이에 상응하는 Encoder 에서의 attention을 계산해내기 때문에
      #       -> 이때, 'encoder_output' 인자와 'encoder_mask' 인자가 필요하다 ?
      out = self.multi_head_attention_layer(query=out, key=encoder_output, value=encoder_output, mask=encoder_mask)

      out = self.position_wise_feed_forward_layer(x=out)

      return out

- Transformer (수정 2 와 동일)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    # 기존에는 Encoder에서 사용하는 pad mask(src_mask)만이 forward()의 인자로 들어왔다면,
    # 이제는 Decoder에서 사용할 subsequent mask (trg_mask)도 함께 주어진다.

    # forward(인코더의 input, 디코더의 input, 인코더의 mask, 디코더의 mask)
    # 인코더의 mask : 'src_mask' 인자 (= 일반적인 Encoder에서 사용하는 pad mask)
    # 디코더의 mask : 'trg_mask' 인자 (= Decoder에서 사용할 subsequent mask)
    def forward(self, src, trg, src_mask, trg_mask): # (?) x, z 는 각각 뭘까?
                                  # x : src
                                  # z : trg
        encoder_output = self.encoder(src, src_mask)
        out = self.decoder(trg, trg_mask, encoder_output, srg_mask) # encoder의 mask(= src_mask) 까지 함께 넘겨준다 !
        return out

- Transformer's Input (Positional Encoding)

In [ ]:
# Transformer의 input은 [1] 단순한 sentence("token embedding sequence")에 더해 [2] "Positional Encoding"이 추가되게 된다.
# 전체 TransformerEmbedding은 [1]"단순 Embedding"과 [2]"PositionalEncoding"의 sequential이다.


class TransformerEmbedding(nn.Module):
    def __init__(self, embedding, positional_encoding):
        super(TransformerEmbedding, self).__init__()
        self.embedding = nn.Sequential(embedding, positional_encoding)


    def forward(self, x):
        out = self.embedding(x)
        return out

In [ ]:
# [1] 단순한 sentence("token embedding sequence")

class Embedding(nn.Module):
    def __init__(self, d_embed, vocab):
        super(Embedding, self).__init__()
        self.embedding = nn.Embedding(len(vocab), d_embed) 
        # 임베딩 벡터의 (행,열) 크기를 (vocab 크기, d_embed 차원 크기)로 하여, embedding을 생성해낸다.
        self.vocab = vocab
        self.d_embed = d_embed


    def forward(self, x):
        out = self.embedding(x) * math.sqrt(self.d_embed)
        # 주목할 점은 embedding에도 scaling을 적용한다는 점이다. 
        # forward()에서 √(d_embed) 를 곱해주게 된다.
        return out

In [ ]:
# [2] "Positional Encoding"
class PositionalEncoding(nn.Module):
    def __init__(self, d_embed, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        encoding = torch.zeros(max_seq_len, d_embed)
        position = torch.arange(0, max_seq_len).unsqueeze(1) # -> 그 결과, 차원은 (0, 1, max_seq_len) 이 됨 ?
        div_term = torch.exp(torch.arange(0, d_embed, 2) * -(math.log(10000.0) / d_embed))
        # torch.arange(0, d_embed, 2) 값 : tensor([0, 2, 4, 6, 8, .., +2, ..., (d_embed-1)])
        encoding[:, 0::2] = torch.sin(position * div_term) # 짝수 index
        encoding[:, 1::2] = torch.cos(position * div_term) # 홀수 index


    def forward(self, x):
        # 구현 상에서 주의할 점은 forward() 내에서 생성하는 Variable이 학습이 되지 않도록 requires_grad=False 옵션을 부여해야 한다는 것이다.
        # PositionalEncoding은 학습되는 parameter가 아니기 때문이다.

        out = x + Variable(self.encoding[:, :x.size(1)], requires_grad=False)
        # :x.size(1) -> 모든 행에 대해 수행하는데, input sequence(x)의 열 크기만큼만 자르는 의미?
        out = self.dropout(out)
        return out


- Transformer (수정 3)

In [ ]:
# 이렇게 생성해낸 embedding을 Transformer에 추가해주자.

# src_embed와 trg_embed를 Transformer의 생성자 인자로 추가한다.
# forward() 내부에서 Encoder와 Decoder의 forward()를 호출할 때 각각 "src_embed(src)"", "trg_embed(trg)"와 같이 input을 TransformerEmbedding으로 감싸 변환해준다.

class Transformer(nn.Module):
    def __init__(self, src_embed, trg_embed, encoder, decoder):
        super(Transformer, self).__init__()

        self.src_embed = src_embed # 추가
        self.trg_embed = trg_embed # 추가

        self.encoder = encoder
        self.decoder = decoder



    # 기존에는 Encoder에서 사용하는 pad mask(src_mask)만이 forward()의 인자로 들어왔다면,
    # 이제는 Decoder에서 사용할 subsequent mask (trg_mask)도 함께 주어진다.

    # forward(인코더의 input, 디코더의 input, 인코더의 mask, 디코더의 mask)
    # 인코더의 mask : 'src_mask' 인자 (= 일반적인 Encoder에서 사용하는 pad mask)
    # 디코더의 mask : 'trg_mask' 인자 (= Decoder에서 사용할 subsequent mask)
    def forward(self, src, trg, src_mask, trg_mask): # (?) x, z 는 각각 뭘까?
                                  # x : src
                                  # z : trg

        # 인코딩/디코딩 과정에서 각각 인코더/디코더 용 embedding 으로 src/trg 를 감싸준다 !                          
        encoder_output = self.encoder(self.src_embed(src), src_mask)
        out = self.decoder(self.trg_embed(trg), trg_mask, encoder_output, srg_mask) # encoder의 mask(= src_mask) 까지 함께 넘겨준다 !
        return out

# 4. Generator

```
Decoder의 output이 그대로 Transformer의 최종 output이 되는 것은 아니다 !
추가적인 layer(= Generator)를 거침


Decoder의 output은 (n_batch X seq_len X d_model) 의 shape를 갖는 matrix
이를 vocabulary 를 사용해 "실제 token"으로 대응해 변환가능하도록 차원을 수정해줘야 함
즉, FC Layer를 거치면서 마지막 dimension "d_model" 을 -> "len(vocab)" 으로 변경하도록 구현

이후 softmax(log_softmax 가 성능 더 높음) 함수를 사용해 각 vocabulary에 대한 확률값으로 변환
```





In [ ]:
class Transformer(nn.Module):

    def __init__(self, src_embed, trg_embed, encoder, decoder, fc_layer):
        super(Transformer, self).__init__()
        self.src_embed = src.embed
        self.trg_embed = trg_embed
        self.encoder = encoder
        self.decoder = decoder
        self.fc_layer = fc_layer


    def forward(self, src, trg, src_mask, trg_mask):
        encoder_output = self.encoder(self.src_embed(src), src_mask)
        out = self.decoder(self.trg_embed(trg), trg_mask, encoder_output, src_mask)
        
        out = self.fc_layer(out) # FC Layer를 통해 마지막 차원을 "d_model" -> "len(vocab)" 으로 변경해줌
        out = F.log_softmax(out, dim=-1) # "dim=-1" 의 의미 ?
                                          # : 마지막 dimension을 기준으로 log_softmax 함수를 통해 계산 ?
                                          # : 마지막 dimension(= "len(vocab)"")에 대한 확률값을 구해야 함

        return out

- Make Model

In [ ]:
# Transformer를 생성하는 예제 함수 make_model()
def make_model(
      src_vocab,
      trg_vocab,
      d_embed = 512,
      n_layer = 6,
      d_model = 512,
      h = 8,
      d_ff = 2048):


    cp = lambda x: copy.deepcopy(x)
    # (?) cp 는 함수인 것 ?!


    # multi_head_attention_layer 생성한 뒤 copy해 사용
    multi_head_attention_layer = MultiHeadAttentionLayer(
                                    d_model = d_model, 
                                    h = h, 
                                    qkv_fc_layer = nn.Linear(d_embed, d_model), 
                                    fc_layer = nn.Linear(d_model, d_embed))
                                    

    # position_wise_feed_forward_layer 생성한 뒤 copy해 사용
    position_wise_feed_forward_layer = PositionWiseFeedForward(
                                          first_fc_layer = nn.Linear(d_embed, d_ff), 
                                          second_fc_layer = nn.Linear(d_ff, d_embed))
    

    # norm_layer 생성한 뒤 copy해 사용
    norm_layer = nn.LayerNorm(d_embed, eps=1e-6)


    # 실제 model 생성
    model = Transformer(
                ##### <SRC embedding 생성> #####
                # TransfomerEmbedding 클래스 : 전체 TransformerEmbedding은 [1]"단순 Embedding"과 [2]"PositionalEncoding"의 sequential이다.
                src_embed = TransfomerEmbedding(
                                # [1] 단순한 sentence("token embedding sequence")
                                embedding = Embedding(
                                                d_embed = d_embed, 
                                                vocab = src_vocab), 
                                 
                                # [2] "Positional Encoding"
                                positional_encoding = PositionalEncoding(
                                                          d_embed = d_embed), 
                                                          ###max_seq_len=5000) # PositionalEncoding 선언 시 default 값
                                ), 
                        

                ##### <TRG embedding 생성> #####
                # TransfomerEmbedding 클래스 : 전체 TransformerEmbedding은 [1]"단순 Embedding"과 [2]"PositionalEncoding"의 sequential이다.
                trg__embed = TransfomerEmbedding(
                                # [1] 단순한 sentence("token embedding sequence")
                                embedding = Embedding(
                                                d_embed = d_embed, 
                                                vocab = trg_vocab), 
                                 
                                # [2] "Positional Encoding"
                                positional_encoding = PositionalEncoding(
                                                          d_embed = d_embed), 
                                                          ###max_seq_len=5000) # PositionalEncoding 선언 시 default 값
                                ), 
                

                ##### <Encoder 생성> #####
                # Encoder 클래스 : encoder_layer/sub_layer 인자를 n_layer 인자만큼 레이터를 쌓아줌
                encoder = Encoder(
                                              # EncoderLayer 클래스 : Encoder Layer 는 크게 Multi-Head Attention Layer 과 Position-wise-Feed-Forward Layer 로 구성됨
                              sub_layer = EncoderLayer(
                                                  multi_head_attention_layer = cp(multi_head_attention), 
                                                  position_wise_feed_forward_layer = cp(position_wise_feed_forward_layer),
                                                  norm_layer = cp(norm_layer)), # (?) 위에서는 EncoderLayer 클래스에 norm_layer 인자는 선언 안 되어 있음 ? 
                              n_layer = n_layer), 
                

                ##### <Decoder 생성> #####
                # Decoder 클래스 : decoder_layer/sub_layer 인자를 n_layer 인자만큼 레이터를 쌓아줌
                decoder = Decoder(
                              sub_layer = DecoderLayer(
                                              masked_multi_head_attention_layer = cp(multi_head_attention_layer),
                                              # DecoderLayer 내부 동작에서 "masked_multi_~" 가 돌아가는거지
                                              # 인자로 인풋해줄 때에는 "masked_multi_~" 레이어와 "multi_head_~" 레이어에 둘다 같은 레이어(멀티-헤드 어텐션 레이어)를 넣어준다??
                                              multi_head_attention_layer = cp(multi_head_attention_layer), 
                                              position_wise_feed_forward_layer = cp(position_wise_feed_forward_layer), 
                                              norm_layer = cp(norm_layer)), 

                              n_layer = n_layer),
                              

                ##### <Generator 생성> #####
                # Generator의 FC Layer 생성
                fc_layer = nn.Linear(d_model, len(trg_vocab)))